In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.cluster import KMeans
from numpy import transpose

from sklearn.metrics.pairwise import cosine_similarity
from IPython.core.display import display
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
import re
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize, LabelEncoder

/usr/local/Cellar/jupyterlab/3.0.13/libexec/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
books = pd.read_csv("data/BX-Books.csv")
books.head()

/usr/local/Cellar/jupyterlab/3.0.13/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
interactions = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", encoding = "ISO-8859-1")
interactions = interactions[interactions["Book-Rating"] != 0]
interactions.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


In [4]:
users = pd.read_csv('data/BX-Users.csv', delimiter=';', encoding = 'ISO-8859-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
users.fillna(users["Age"].mean(), inplace=True)

## Реализовать персональный топ - принимает на вход возраст и локацию, на выходе персональный топ

In [6]:
users["Country"] = users.apply(lambda x: x['Location'].split(", ")[-1], axis = 1)

In [7]:
def personal_recommendation(age, country, top=10):
    if age <= 16:
        condition = users["Age"].between(0, 16)
    elif age <= 65:
        condition = users["Age"].between(17, 65)
    else:
        condition = users["Age"].between(66, 150)

    same_users = pd.DataFrame(
        users[(condition) & (users["Country"] == country)]["User-ID"]
    )

    user_interactions = interactions.merge(same_users, on="User-ID", how="inner")
    user_interactions = (
        user_interactions.groupby("ISBN")
        .agg({"User-ID": "count", "Book-Rating": "mean"})
        .sort_values(["Book-Rating", "User-ID"], ascending=False)
    )

    user_interactions = user_interactions[user_interactions["User-ID"] > 5]
    display(
        (
            user_interactions[:top]
            .merge(books, on="ISBN", how="inner")
            .drop(
                columns=["ISBN", "User-ID", "Image-URL-S", "Image-URL-M", "Image-URL-L"]
            )
        )
    )


In [8]:
personal_recommendation(45, 'usa')

,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,10.000000,The Sneetches and Other Stories,Dr. Seuss,1961,Random House Children's Books
1,10.000000,"Oh, the Thinks You Can Think! (I Can Read It A...",Dr. Seuss,1975,Random House Children's Books
2,10.000000,The Lord of the Rings (Leatherette Collector's...,J. R. R. Tolkien,1974,Houghton Mifflin Company
3,10.000000,Goodnight Moon Board Book,Margaret Wise Brown,1991,HarperFestival
4,9.916667,Calvin and Hobbes,Bill Watterson,1987,Andrews McMeel Publishing
5,9.888889,Dilbert: A Book of Postcards,Scott Adams,1996,Andrews McMeel Pub
6,9.875000,TOLKIEN MAGNETIC POSTCARDS(tm) 12 Full-color M...,Brothers Hildebrandt,2001,Running Press Book Publishers
7,9.857143,Diary of a Young Girl: The Definitive Editon,Anne Frank,1996,Anchor
8,9.833333,The Giving Tree,Shel Silverstein,1964,HarperCollins Publishers


In [9]:
personal_recommendation(16, 'usa')

,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,9.857143,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic
1,9.769231,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic
2,9.666667,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey
3,9.625000,The Bad Beginning (A Series of Unfortunate Eve...,Lemony Snicket,1999,HarperCollins
4,9.625000,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
5,9.333333,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic
6,9.272727,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic
7,9.157895,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
8,9.090909,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic
9,9.000000,The Giver (21st Century Reference),LOIS LOWRY,1994,Laure Leaf


In [10]:
personal_recommendation(70, 'usa')

,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,9.166667,Angels &amp; Demons,Dan Brown,2001,Pocket Star
1,8.937500,The Da Vinci Code,Dan Brown,2003,Doubleday
2,7.666667,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
3,7.571429,The Summons,John Grisham,2002,Dell Publishing Company
4,7.166667,The Rainmaker,JOHN GRISHAM,1996,Dell
5,3.555556,Wild Animus,Rich Shapero,2004,Too Far


## На основе метода кластеризации похожих пользователей построить рекомендации

In [11]:
books_grouped = (
    interactions.groupby("ISBN")["User-ID"]
    .count()
    .reset_index()
    .rename(columns={"User-ID": "users_count"})
)
user_grouped = (
    interactions.groupby("User-ID")["ISBN"]
    .count()
    .reset_index()
    .rename(columns={"ISBN": "books_count"})
)

interactions_megred = interactions.merge(books_grouped, on=["ISBN"]).merge(
    user_grouped, on=["User-ID"]
)

interactions_megred = interactions_megred[
    (interactions_megred["users_count"] > 5)
    & (interactions_megred["books_count"] > 5)
]

interactions_megred = interactions_megred.merge(
    books[["ISBN", "Book-Title"]],
    on=["ISBN"],
)


In [12]:
interactions_megred

,User-ID,ISBN,Book-Rating,users_count,books_count,Book-Title
0,86583,3404139178,9,8,18,Das Lacheln der Fortuna: Historischer Roman
1,132500,3404139178,10,8,43,Das Lacheln der Fortuna: Historischer Roman
2,66483,3404139178,10,8,83,Das Lacheln der Fortuna: Historischer Roman
3,276866,3404139178,9,8,11,Das Lacheln der Fortuna: Historischer Roman
4,106534,3404139178,6,8,6,Das Lacheln der Fortuna: Historischer Roman
...,...,...,...,...,...,...
129987,84867,3453177746,9,9,13,Hannibal
129988,183213,3499137909,10,7,8,Der Plan von der Abschaffung des Dunkels.
129989,140226,3499137909,5,7,10,Der Plan von der Abschaffung des Dunkels.
129990,206652,1860465811,9,8,10,Wind-Up Bird Chronicle


In [13]:
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()
interactions_megred["user_id"] = user_encoder.fit_transform(interactions_megred["User-ID"])
interactions_megred["book_id"] = book_encoder.fit_transform(interactions_megred["ISBN"])

In [14]:
sparse_matrix = coo_matrix(
    (
        interactions_megred["Book-Rating"],
        (interactions_megred["user_id"], interactions_megred["book_id"]),
    ),
    shape=(
        len(interactions_megred["user_id"].unique()),
        len(interactions_megred["book_id"].unique()),
    ),
)
sparse_matrix = normalize(sparse_matrix)

In [15]:
k_means = KMeans(n_clusters=5, n_jobs=-1)

In [16]:
labels = k_means.fit_predict(sparse_matrix)

/usr/local/Cellar/jupyterlab/3.0.13/libexec/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


In [17]:
labels

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [18]:
users_clusters = {}
for _id, label in enumerate(labels):
    users_clusters.setdefault(label, []).append(_id)

In [19]:
for cluster, users in users_clusters.items():
    print(cluster, len(users))

1 10287
4 165
0 261
2 229
3 163


In [20]:
def get_users_top(input_users, top=10, book_rate=False):
    users = pd.DataFrame(data=input_users, columns=["user_id"])
    current_interactions = (
        interactions_megred.merge(users, on="user_id", how="inner")
        .groupby("ISBN")
        .agg({"Book-Rating": "mean", "User-ID": "count"})
        .sort_values(
            ["User-ID", "Book-Rating"] if not book_rate else ["Book-Rating", "User-ID"],
            ascending=False,
        )
    )
    return (
        current_interactions[:top]
        .merge(books, on="ISBN", how="inner")
        .drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )


In [21]:
for cluster, users in users_clusters.items():
    print(f'Top for cluster (without rating sort): {cluster}')
    display(get_users_top(users))
    print(f'Top for cluster (with rating sort): {cluster}')
    display(get_users_top(users, book_rate=True))

Top for cluster (without rating sort): 1


,Book-Rating,User-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,4.050459,218,Wild Animus,Rich Shapero,2004,Too Far
1,8.964467,197,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
2,7.647727,176,Interview with the Vampire,Anne Rice,1993,Ballantine Books
3,7.932515,163,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
4,8.198675,151,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books
5,9.061224,147,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
6,7.815068,146,The Firm,John Grisham,1992,Bantam Dell Publishing Group
7,7.800000,140,Jurassic Park,Michael Crichton,1999,Ballantine Books
8,8.119403,134,Angels &amp; Demons,Dan Brown,2001,Pocket Star
9,7.664179,134,Snow Falling on Cedars,David Guterson,1995,Vintage Books USA


Top for cluster (with rating sort): 1


,Book-Rating,User-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,10.0,10,Dilbert: A Book of Postcards,Scott Adams,1996,Andrews McMeel Pub
1,10.0,10,Postmarked Yesteryear: 30 Rare Holiday Postcards,Pamela E. Apkarian-Russell,2001,Collectors Press
2,10.0,6,"Oh, the Thinks You Can Think! (I Can Read It A...",Dr. Seuss,1975,Random House Children's Books
3,10.0,6,The Lord of the Rings,J. R. R. Tolkien,2002,Houghton Mifflin Company
4,10.0,6,Six by Seuss: A Treasury of Dr. Seuss Classics,Seuss,1991,Random House Children's Books
5,10.0,6,Natural California: A Postcard Book,Not Applicable (Na ),1990,Running Pr
6,10.0,5,The Lord of the Rings (Leatherette Collector's...,J. R. R. Tolkien,1974,Houghton Mifflin Company
7,10.0,5,The Hobbit,J. R. R. Tolkien,1988,Houghton Mifflin Co
8,10.0,5,Uncle John's Supremely Satisfying Bathroom Rea...,Bathroom Readers Institute,2001,Bathroom Reader Press
9,10.0,4,"The Lord of the rings,",J. R. R Tolkien,1968,Allen and Unwin


Top for cluster (without rating sort): 4


...

,Book-Rating,User-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,10.0,4,Watership Down,Richard Adams,1976,Avon
1,10.0,3,La Sombra del Viento,Carlos Ruiz Zafon,2003,Planeta
2,10.0,2,The God of Small Things,Arundhati Roy,1998,Perennial
3,10.0,2,My Year of Meats,Ruth L. Ozeki,1999,Penguin Books
4,10.0,2,Ender's Shadow (Ender Wiggins Saga (Hardcover)),Orson Scott Card,1999,Tor Books
5,10.0,2,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey
6,10.0,2,The Princess Bride: S Morgenstern's Classic Ta...,WILLIAM GOLDMAN,1987,Del Rey
7,10.0,2,The Hitchhiker's Guide to the Galaxy,Douglas Adams,1995,Ballantine Books
8,10.0,2,Monkeewrench,P. J. Tracy,2003,Putnam Publishing Group
9,10.0,2,"Remember When (Roberts, Nora)",Nora Roberts,2003,Putnam Publishing Group


Top for cluster (without rating sort): 3


,Book-Rating,User-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,8.644172,163,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
1,8.615385,13,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial
2,7.846154,13,Girl with a Pearl Earring,Tracy Chevalier,2001,Plume Books
3,8.083333,12,The Pilot's Wife : A Novel,Anita Shreve,1999,Back Bay Books
4,8.272727,11,The Hours: A Novel,Michael Cunningham,2002,Picador
5,7.909091,11,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books
6,7.700000,10,The Da Vinci Code,Dan Brown,2003,Doubleday
7,8.777778,9,"Tuesdays with Morrie: An Old Man, a Young Man,...",MITCH ALBOM,1997,Doubleday
8,8.375000,8,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
9,8.375000,8,Good in Bed,Jennifer Weiner,2002,Washington Square Press


Top for cluster (with rating sort): 3


,Book-Rating,User-ID,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,10.0,2,The Purpose-Driven Life: What on Earth Am I He...,Rick Warren,2002,Zondervan Publishing Company
1,10.0,2,Seabiscuit,LAURA HILLENBRAND,2003,Ballantine Books
2,10.0,2,Red Dragon,Thomas Harris,2000,Dell Publishing Company
3,10.0,2,To Kill a Mockingbird,Harper Lee,1985,Warner Books
4,10.0,2,The Stand: Complete and Uncut,Stephen King,1991,Signet Book
5,10.0,2,A Midsummer Nights Dream (Bantam Classic),William Shakespeare,1988,Bantam
6,10.0,2,The Secret Life of Bees,Sue Monk Kidd,2002,Viking Books
7,10.0,2,The Ultimate Weight Solution: The 7 Keys to We...,Phil McGraw,2003,Free Press
8,10.0,1,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
9,10.0,1,The Boy Next Door,Meggin Cabot,2002,Avon Trade


## Применить методы расмотренные на лекции

### Совстречаемость

In [22]:
user_books = (interactions_megred[["user_id", "book_id"]]
                 .groupby(["user_id"])["book_id"]
                 .apply(list).reset_index()
                 .rename(columns={"book_id": "book_ids"}))
user_books["books_count"] = user_books["book_ids"].map(len)
user_books = user_books[user_books["books_count"] > 1]
user_books

,user_id,book_ids,books_count
1,1,"[1393, 5417, 5107, 1386, 1710]",5
2,2,"[5066, 5069, 5631, 7389, 5016, 6959]",6
3,3,"[10388, 6532, 10536, 10363]",4
4,4,"[4546, 1715, 5164, 9923, 2546, 3481, 4032, 499...",16
5,5,"[9930, 9923, 421, 1198, 9071, 415, 6252, 2661]",8
...,...,...,...
11099,11099,"[1690, 4489, 1151, 426, 981, 1633, 3479, 4103,...",28
11101,11101,"[8637, 5688, 8571, 6114]",4
11102,11102,"[8940, 7039, 1151, 3748, 8807, 981, 4354, 7883...",12
11103,11103,"[4609, 9666, 9914, 4215]",4


In [23]:
user_books.values

array([[1, list([1393, 5417, 5107, 1386, 1710]), 5],
       [2, list([5066, 5069, 5631, 7389, 5016, 6959]), 6],
       [3, list([10388, 6532, 10536, 10363]), 4],
       ...,
       [11102,
        list([8940, 7039, 1151, 3748, 8807, 981, 4354, 7883, 111, 240, 3220, 6510]),
        12],
       [11103, list([4609, 9666, 9914, 4215]), 4],
       [11104, list([4020, 6883, 2647]), 3]], dtype=object)

In [24]:
book_ratings = (
    interactions_megred[["book_id", "Book-Rating"]]
    .groupby("book_id")["Book-Rating"]
    .mean()
    .to_dict()
)

In [25]:
cooc = {}
for i in user_books.values:
    for j in range(len(i[1])):
        for k in range(len(i[1])):
            if j == k: continue
            key = str(i[1][j]) + "_" + str(i[1][k])
            cooc[key] = cooc.setdefault(key, 0) + (book_ratings[i[1][j]] + book_ratings[i[1][k]]) / 2

In [26]:
cooc_list = []
for i, j in cooc.items():
    cooc_list.append(i.split("_") + [j])
cooc_rec = pd.DataFrame(cooc_list, columns=["book_1", "book_2", "measure"])
cooc_rec

,book_1,book_2,measure
0,1393,5417,7.477273
1,1393,5107,7.272727
2,1393,1386,7.005051
3,1393,1710,6.727273
4,5417,1393,7.477273
...,...,...,...
12047337,4215,9666,7.892857
12047338,4020,2647,6.683499
12047339,6883,2647,6.972973
12047340,2647,4020,6.683499


In [27]:
cooc_rec[cooc_rec['book_1'] == '7039'].sort_values("measure", ascending=False)

,book_1,book_2,measure
174197,7039,4218,553.339290
167964,7039,4227,386.898585
71079,7039,4235,324.868885
898337,7039,4228,283.271186
71059,7039,4229,243.288345
...,...,...,...
11347624,7039,2351,6.788208
7931942,7039,4552,6.788208
11927346,7039,9771,6.710430
899164,7039,3432,6.488208


In [28]:
def get_cooc_recommendation(book_id, top=10):
    encode_id = book_encoder.transform([book_id])[0]
    rec_books = cooc_rec[cooc_rec["book_1"] == str(encode_id)].sort_values(
        "measure", ascending=False
    )
    rec_books["book_2"] = rec_books.apply(
        lambda x: book_encoder.inverse_transform([int(x[1])])[0], axis=1
    )
    book = books[books["ISBN"] == book_id].values[0]
    display(
        (
            rec_books[:top]
            .merge(books, left_on="book_2", right_on="ISBN", how="inner")
            .drop(
                columns=[
                    "ISBN",
                    "Image-URL-S",
                    "Image-URL-M",
                    "Image-URL-L",
                    "book_1",
                    "book_2",
                ]
            )
        )
    )


In [29]:
get_cooc_recommendation("0439136369")

,measure,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,429.410577,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic
1,386.898585,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
2,307.084800,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2002,Scholastic Paperbacks
3,244.224777,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
4,155.703112,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic
5,107.534260,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic
6,75.181680,Interview with the Vampire,Anne Rice,1993,Ballantine Books
7,71.283019,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey
8,68.015770,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
9,63.917389,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,2001,Scholastic


In [30]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


### Content-based

In [31]:
texts = books.apply(lambda x: " ".join(map(str, x[1:5])), axis=1)

In [32]:
texts

0         Classical Mythology Mark P. O. Morford 2002 Ox...
1         Clara Callan Richard Bruce Wright 2001 HarperF...
2         Decision in Normandy Carlo D'Este 1991 HarperP...
3         Flu: The Story of the Great Influenza Pandemic...
4         The Mummies of Urumchi E. J. W. Barber 1999 W....
                                ...                        
271355    There's a Bat in Bunk Five Paula Danziger 1988...
271356    From One to One Hundred Teri Sloat 1991 Dutton...
271357    Lily Dale : The True Story of the Town that Ta...
271358    Republic (World's Classics) Plato 1996 Oxford ...
271359    A Guided Tour of Rene Descartes' Meditations o...
Length: 271360, dtype: object

In [33]:
book_description = [simple_preprocess(re.sub("[^A-Za-z]+", " ", i.lower())) for i in texts]

In [34]:
num_features = 100
model = Word2Vec(book_description, vector_size=num_features, min_count=1, workers=10)

In [35]:
def avg_feature_vector(words, model, num_features, index2word_set):
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = feature_vec / n_words
    return feature_vec

In [36]:
books_embs = np.zeros((len(book_description), num_features))
index2word_set = set(model.wv.index_to_key)
for i in range(books_embs.shape[0]):
    books_embs[i] = avg_feature_vector(book_description[i], model, num_features, index2word_set)

In [37]:
def get_rec_I2I(i):
    book = books.iloc[i]
    metrics = cosine_similarity([books_embs[i]], books_embs)
    print("Для книги", book["Book-Title"], "такие рекомендации:")
    display(
        (
            books.iloc[np.argsort(metrics)[0][::-1][1:11]].drop(
                columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"]
            )
        )
    )


In [38]:
get_rec_I2I(198793)

Для книги The Lion, the Witch and the Wardrobe (The Chronicles of Narnia, Book 2) такие рекомендации:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
9727,"The Lion, the Witch, and the Wardrobe (The Chr...",C. S. Lewis,1994,HarperCollins
266203,The Hallowed Isle : The Book of the Cauldron a...,Diana L. Paxson,2001,Eos
237348,The Hallowed Isle : The Book Of The Sword And ...,Diana L. Paxson,2000,Eos
96189,The Chronicles of Narnia: The Magician's Nephe...,C. S. Lewis,1994,HarperCollins
131861,"The Lion, the Witch and the Wardrobe (The Chro...",C. S. Lewis,1994,HarperCollins
125409,"The Lion, the Witch and the Wardrobe (Book 1 i...",C. S. Lewis,1987,Scholastic
251618,"The Lion, the Witch and the Wardrobe",C.S. Lewis,1998,Collins
229900,"The Way of Merlin: The Prophet, the Goddess an...",R.J. Stewart,1992,Natl Book Network
129033,"The Lady of the Sorrows (The Bitterbynde, Book 2)",Cecilia Dart-Thornton,2002,Aspect
266269,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,2001,Recorded Books


In [39]:
get_rec_I2I(5431)

Для книги Harry Potter and the Goblet of Fire (Book 4) такие рекомендации:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
3839,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic
6330,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic
10440,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2002,Scholastic
5432,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic
3459,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic
79370,Harry Potter and the Chamber of Secrets Postca...,J. K. Rowling,2002,Scholastic
203712,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
86390,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
5506,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
6932,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2002,Scholastic Paperbacks


In [40]:
def get_rec_U2I(i):
    current_user_books = user_books.iloc[i]["book_ids"]
    current_user_books = (
        pd.DataFrame({"ISBN": book_encoder.inverse_transform(current_user_books)})
        .merge(books, on="ISBN", how="inner")
        .drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )
    user_vector = []
    texts = current_user_books.apply(lambda x: " ".join(map(str, x[1:4])), axis=1)
    for i in texts:
        user_vector += simple_preprocess(re.sub("[^A-Za-z]+", " ", i.lower()))
    user_emb = avg_feature_vector(user_vector, model, num_features, index2word_set)
    metrics = cosine_similarity([user_emb], books_embs)
    print("Для пользователя, который купил книги")
    display(current_user_books)
    print("Такие рекомендации")
    display(
        (
            books.iloc[np.argsort(metrics)[0][::-1][1:11]].drop(
                columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"]
            )
        )
    )

In [41]:
get_rec_U2I(5319)

Для пользователя, который купил книги


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,The Testament,John Grisham,1999,Dell
1,POSTCARDS,Annie Proulx,1994,Scribner
2,The All-True Travels and Adventures of Lidie N...,Jane Smiley,1999,Ballantine Books
3,The Partner,John Grisham,1998,Dell Publishing Company
4,A Time to Kill,JOHN GRISHAM,1992,Dell
5,Back When We Were Grownups : A Novel (Ballanti...,ANNE TYLER,2002,Ballantine Books
6,The Last Suppers,DIANE MOTT DAVIDSON,1995,Crimeline
7,Horse Heaven (Ballantine Reader's Circle),Jane Smiley,2001,Ballantine Books
8,If Morning Ever Comes : A Novel,ANNE TYLER,1996,Ballantine Books


Такие рекомендации


,Book-Title,Book-Author,Year-Of-Publication,Publisher
271081,Scarlet Ruse,John D Macdonald,1973,Ballantine Books
239241,Scarlet Ruse,John Macdonald,1982,Ballantine Books
230764,Empty Copper Sea,John D Macdonald,1979,Ballantine Books
4040,Barrier Island,John D. MacDonald,1991,Fawcett Books
23433,Dreadful Lemon Sky,John D. MacDonald,1987,Ballantine Books
160773,Dreadful Lemon Sky,John Macdonald,1975,Ballantine Books
47017,Green Ripper,John D Macdonald,1981,Ballantine Books
262850,Turquoise Lament,John Macdonald,1974,Ballantine Books
3896,Midnight Voices,JOHN SAUL,2003,Ballantine Books
17299,Midnight Voices,JOHN SAUL,2002,Ballantine Books


In [42]:
get_rec_U2I(421)

Для пользователя, который купил книги


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Obasan,Joy Kogawa,1983,Penguin Books
1,Selected Poems (Dover Thrift Edition),Emily Dickinson,1990,Dover Publications
2,Bachelor Brothers' Bed &amp; Breakfast,Bill Richardson,1997,St. Martin's Griffin
3,Violet &amp; Claire,Francesca Lia Block,2000,Harpercollins Juvenile Books
4,Notes from the Underground (Dover Thrift Editi...,Fyodor Dostoyevsky,1992,Dover Publications


Такие рекомендации


,Book-Title,Book-Author,Year-Of-Publication,Publisher
490,Solv-A-Crime Puzzles (Dover Game and Puzzle Ac...,A. C. Gordon,1995,Dover Publications
5123,Margaret the Queen (Coronet Books),Nigel Tranter,1998,Coronet Books &amp; Publications
61634,Silly Jokes (Dover Little Activity Books),Victoria Fremont,1991,Dover Publications
40881,Six Japanese Print Postcards (Small-Format Car...,Utamaro,1994,Dover Publications
232182,Ugly Duckling Sticker Storybook (Dover Little ...,Marty Noble,1997,Dover Publications
186106,Soap Bubbles (Dover Science Books),Charles V. Boys,1959,Dover Publications
244913,Dot-To-Dot (Beginners Activity Books),Anna Pomaska,1997,Dover Publications
196943,Wood-Frame House Construction,Gerald E. Sherwood,1990,Dover Publications
185996,"Riddles, Riddles, Riddles (Dover Game &amp; Pu...",Darwin A. Hindman,1997,Dover Publications
250045,Sixpenny Girl,Meg Hutchinson,2004,Coronet Books &amp; Publications


### Коллаборативная фильтрация

In [43]:
sparse_matrix = coo_matrix(
    (
        interactions_megred["Book-Rating"],
        (interactions_megred["user_id"], interactions_megred["book_id"]),
    ),
    shape=(
        len(interactions_megred["user_id"].unique()),
        len(interactions_megred["book_id"].unique()),
    ),
)

In [44]:
sparse_matrix.shape

(11105, 10724)

In [45]:
books_num = (
    (
        interactions_megred[["user_id", "book_id"]]
        .groupby(["user_id"])["book_id"]
        .apply(list)
        .reset_index()
        .rename(columns={"book_id": "book_ids"})
    )["book_ids"]
    .map(len)
    .to_numpy()
    .reshape(-1, 1)
)

In [46]:
books_num.shape

(11105, 1)

In [47]:
avg_rating = np.array(np.sum(sparse_matrix, axis=1) / books_num)

In [48]:
users_rating = interactions_megred.apply(
    lambda x: x["Book-Rating"] - avg_rating[x["user_id"]][0], axis=1
)

In [49]:
sparse_matrix_users_rating = coo_matrix(
    (
        users_rating,
        (interactions_megred["user_id"], interactions_megred["book_id"]),
    ),
    shape=(
        len(interactions_megred["user_id"].unique()),
        len(interactions_megred["book_id"].unique()),
    ),
)

In [50]:
books_rating = interactions_megred.apply(
    lambda x: x["Book-Rating"] - book_ratings[x["book_id"]], axis=1
)

In [51]:
sparse_matrix_books_rating = coo_matrix(
    (
        books_rating,
        (interactions_megred["user_id"], interactions_megred["book_id"]),
    ),
    shape=(
        len(interactions_megred["user_id"].unique()),
        len(interactions_megred["book_id"].unique()),
    ),
)

In [52]:
def get_user_based_rec(idx):
    user_rates = sparse_matrix.getrow(idx).toarray()[0]
    watched_books = np.where(user_rates != 0)[0]
    metrics = cosine_similarity([user_rates], sparse_matrix).reshape(-1, 1)
    rates = csr_matrix.multiply(sparse_matrix_users_rating.copy(), metrics)
    rates = (1 - user_rates.astype(bool)) * np.array(np.sum(rates, axis=0))[0]
    print("Для пользователя, который покупал такие книги:")
    display(
        books.merge(
            pd.DataFrame({"ISBN": book_encoder.inverse_transform(watched_books)}),
            on="ISBN",
            how="inner",
        ).drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )
    print("Такие рекомендации: ")
    display(
        books.merge(
            pd.DataFrame(
                {"ISBN": book_encoder.inverse_transform(np.argsort(rates)[::-1][0:10])}
            ),
            on="ISBN",
            how="inner",
        ).drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )


In [53]:
get_user_based_rec(43)

Для пользователя, который покупал такие книги:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Notes From a Small Planet,Bill Bryson,0,Bantam Doubleday Dell
1,Northern Lights (His Dark Materials S.),Philip Pullman,1998,Scholastic Point


Такие рекомендации: 


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Angelas Ashes,Frank Mccourt,0,Harpercollins Uk
1,"The Subtle Knife (His Dark Materials, Book 2)",Philip Pullman,1998,Scholastic Point
2,Blackberry Wine,Joanne Harris,2001,Black Swan
3,Prodigal Summer: A Novel,Barbara Kingsolver,2001,Perennial
4,Route 66 Postcards: Greetings from the Mother ...,Michael Wallis,1993,St. Martin's Press
5,Bloodsucking Fiends: A Love Story,Christopher Moore,1996,Perennial
6,Count Karlstein,Philip Pullman,2000,Yearling Books
7,Ex Libris : Confessions of a Common Reader,Anne Fadiman,2000,"Farrar, Straus and Giroux"
8,"The New York Trilogy: City of Glass, Ghosts, t...",Paul Auster,1994,Penguin Books
9,Wizard of Oz Postcards in Full Color (Card Books),Ted Menten,1986,Dover Publications


In [54]:
get_user_based_rec(5319)

Для пользователя, который покупал такие книги:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,A Prayer for Owen Meany,John Irving,1990,Ballantine Books
1,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic
2,God Save the Sweet Potato Queens,JILL CONNER BROWNE,2001,Three Rivers Press
3,The Sweet Potato Queens' Book of Love,JILL CONNER BROWNE,1999,Three Rivers Press
4,The Scandalous Summer of Sissy LeBlanc : A Novel,Loraine Despres,2002,Perennial
5,Night Shift,Stephen King,1994,Signet Book
6,Raney,Clyde Edgerton,1992,Ballantine Books


Такие рекомендации: 


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,The Bean Trees,Barbara Kingsolver,1989,Perennial
1,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
2,The Da Vinci Code,Dan Brown,2003,Doubleday
3,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
4,"Tuesdays with Morrie: An Old Man, a Young Man,...",MITCH ALBOM,1997,Doubleday
5,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic
6,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic
7,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic
8,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic
9,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2002,Scholastic Paperbacks


In [55]:
def get_item_based_rec(idx):
    user_rates = sparse_matrix_books_rating.getrow(idx).toarray()[0]
    watched_books = np.where(user_rates != 0)[0]
    metrics = cosine_similarity([user_rates], sparse_matrix_users_rating).reshape(-1, 1)
    rates = csr_matrix.multiply(sparse_matrix_users_rating.copy(), metrics)
    total_rate = np.array(np.sum(rates, axis=1)).ravel()
    print("Для пользователя, который покупал такие книги:")
    display(
        pd.DataFrame({"ISBN": book_encoder.inverse_transform(watched_books)})
        .merge(books, on="ISBN", how="inner")
        .drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )
    print("Такие рекомендации: ")
    display(
        pd.DataFrame(
            {"ISBN": book_encoder.inverse_transform(np.argsort(total_rate)[::-1][:10])}
        )
        .merge(books, on="ISBN", how="inner")
        .drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )

In [56]:
get_item_based_rec(43)

Для пользователя, который покупал такие книги:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Notes From a Small Planet,Bill Bryson,0,Bantam Doubleday Dell
1,Northern Lights (His Dark Materials S.),Philip Pullman,1998,Scholastic Point


Такие рекомендации: 


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,A Season Beyond a Kiss,Kathleen E. Woodiwiss,2000,Avon Trade
1,Animal Husbandry,Laura Zigman,2001,Delta
2,How Stella Got Her Groove Back,Terry McMillan,1998,Signet Book
3,A Prayer for Owen Meany (Ballantine Reader's C...,John Irving,1997,Ballantine Books
4,"Roll of Thunder, Hear My Cry",Mildred D. Taylor,1984,Starfire
5,Lady Chatterley's Lover,D.H. Lawrence,1969,Penguin Books Ltd
6,Whispers,BELVA PLAIN,1994,Dell
7,The Defiant Hero,SUZANNE BROCKMANN,2003,Ivy Books
8,Green Eggs and Ham (I Can Read It All by Mysel...,Dr. Seuss,1960,Random House Books for Young Readers
9,The Cat Who Went Up the Creek,Lilian Jackson Braun,2003,Jove Books


In [57]:
get_item_based_rec(5319)

Для пользователя, который покупал такие книги:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,The Scandalous Summer of Sissy LeBlanc : A Novel,Loraine Despres,2002,Perennial
1,Raney,Clyde Edgerton,1992,Ballantine Books
2,A Prayer for Owen Meany,John Irving,1990,Ballantine Books
3,Night Shift,Stephen King,1994,Signet Book
4,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic
5,The Sweet Potato Queens' Book of Love,JILL CONNER BROWNE,1999,Three Rivers Press
6,God Save the Sweet Potato Queens,JILL CONNER BROWNE,2001,Three Rivers Press


Такие рекомендации: 


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,It's Not About the Bike: My Journey Back to Life,Lance Armstrong,2001,Berkley Publishing Group
1,Twelve Across,Barbara Delinsky,2000,Mira
2,Pink Slip Party,Cara Lockwood,2004,Downtown Press
3,Vixen 03,Clive Cussler,1984,Bantam Books
4,Bridget Jones's Diary,Helen Fielding,1997,Picador (UK)
5,Cat's Cradle,Kurt Vonnegut,1997,Dell Publishing Company
6,The Beach House,James Patterson,2002,"Little, Brown"
7,The Fiery Cross,Diana Gabaldon,2001,Delacorte Press
8,Brothers in Arms,Lois McMaster Bujold,1989,Baen
9,Night Prey,John Sandford,1994,Putnam Pub Group


### Матричная Факторизация

In [58]:
sparse_matrix_norm = normalize(sparse_matrix)
U, S, V = svds(sparse_matrix_norm.astype(float), 150)
user_embs = U
items_embs = V.T

In [59]:
def get_matrix_I2I(idx):
    book = books[books["ISBN"] == book_encoder.inverse_transform([idx])[0]].iloc[0]
    metrics = cosine_similarity([items_embs[idx]], items_embs)
    print("Для книги", book["Book-Title"], "такие рекомендации:")
    display(
        books.merge(
            pd.DataFrame(
                {
                    "ISBN": book_encoder.inverse_transform(
                        np.argsort(metrics)[0][::-1][1:11]
                    )
                }
            ),
            on="ISBN",
        ).drop(columns=["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
    )

In [60]:
get_matrix_I2I(5611)

Для книги Hard Evidence такие рекомендации:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Thorns of Truth,Eileen Goudge,1999,Signet Book
1,At Home in Mitford (The Mitford Years),Jan Karon,1996,Viking Books
2,Out to Canaan (The Mitford Years),Jan Karon,1998,Viking Books
3,"These High, Green Hills (The Mitford Years)",Jan Karon,1997,Viking Books
4,A Light in the Window (The Mitford Years),Jan Karon,1996,Penguin Books
5,The Matarese Countdown,Robert Ludlum,1998,Bantam Books
6,A New Song (Mitford Years (Paperback)),Jan Karon,2000,Viking Books
7,The Present: A Malory Holiday Novel (Malory No...,Johanna Lindsey,1998,Avon Books
8,"Eeyore, Be Happy (Little Golden Book)",Don Ferguson,1996,Western Publishing Company
9,Birds of Prey: A Novel of Suspense,J.A. Jance,2001,William Morrow


In [63]:
get_matrix_I2I(404)

Для книги The Lone Ranger and Tonto Fistfight in Heaven такие рекомендации:


,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002,Ballantine Books
1,"The Game of Kings (Lymond Chronicles, 1)",DOROTHY DUNNETT,1997,Vintage
2,Tracks RI,Louise Erdrich,2004,Perennial
3,The English Patient,Michael Ondaatje,1996,Vintage Books USA
4,Victory Over Japan : A Book of Stories,Ellen Gilchrist,1985,Back Bay Books
5,Of Human Bondage (Bantam Classic),W. Somerset Maugham,1991,Bantam Classics
6,Touch,Elmore Leonard,1988,HarperCollins Publishers
7,Son of the Shadows (The Sevenwaters Trilogy),Juliet Marillier,2002,Tor Fantasy
8,Enigma,Robert Harris,1995,Random House Inc
9,King Lear (3rd Series),William Shakespeare,1997,Arden Shakespeare
